In [1]:
import sys
sys.path.append('../')
import os
os.environ['REGISTRY_PATH']='../registry.pb'
os.environ['FEATURE_FOLDER_PATH']='../features'
from pyspark.sql import SparkSession

import pandas as pd
from datetime import datetime

In [2]:
from feature_repo import store

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/21 11:38:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/code/contrib/feast_epam/env/lib/python3.9/site-packages/feast/infra/offline_stores/contrib/spark_offline_store/spark_source.py:67: RuntimeWarning: The spark data source API is an experimental feature in alpha development. This API is unstable and it could and most probably will be changed in the future.
  warnings.warn(


In [3]:
spark=SparkSession.getActiveSession()

In [4]:
f_store=store.get_store()

/opt/code/contrib/feast_epam/env/lib/python3.9/site-packages/feast/repo_config.py:228: RuntimeWarning: `entity_key_serialization_version` is either not specified in the feature_store.yaml, or is specified to a value <= 1.This serialization version may cause errors when trying to write fields with the `Long` data type into the online store. Specifying `entity_key_serialization_version` to 2 is recommended for new projects. 
  warnings.warn(


In [5]:
f_store.list_data_sources()[0]

In [6]:
f_store.list_feature_views()[0]

<FeatureView(name = user_info, entities = ['user'], ttl = 0:00:00, stream_source = None, batch_source = {
  "type": "BATCH_SPARK",
  "timestampField": "txn_datetime",
  "dataSourceClassType": "feast.infra.offline_stores.contrib.spark_offline_store.spark_source.SparkSource",
  "name": "user_loan",
  "sparkOptions": {
    "path": "loan_default_gen.parquet",
    "fileFormat": "parquet"
  }
}, entity_columns = [user_id-String], features = [Age-Int64, Education-String, CreditScore-Int64, MonthsEmployed-Int64], description = Calculate overall transaction behaviors of customer, tags = {}, owner = , projection = FeatureViewProjection(name='user_info', name_alias=None, desired_features=[], features=[Age-Int64, Education-String, CreditScore-Int64, MonthsEmployed-Int64], join_key_map={}), created_timestamp = 2024-03-21 01:24:41.682960, last_updated_timestamp = 2024-03-21 01:24:41.682960, online = True, materialization_intervals = [])>

In [5]:
entity_df=pd.DataFrame(
    [
        ['HlWpZydxscnmjGgeIPLi',pd.datetime(2023,1,2)],
        ['RckfsuJrhXFYAToUVlmy',pd.datetime(2024,1,9)],
    ],
    columns=['user_id','event_timestamp']
)

/var/folders/_6/gftk13754td4yhqfcv8mzqlh0000gn/T/ipykernel_49050/3345325806.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  ['HlWpZydxscnmjGgeIPLi',pd.datetime(2023,1,2)],
/var/folders/_6/gftk13754td4yhqfcv8mzqlh0000gn/T/ipykernel_49050/3345325806.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  ['RckfsuJrhXFYAToUVlmy',pd.datetime(2024,1,9)],


In [14]:
df=spark.read.parquet('../features/loan_default_gen')


In [17]:
df.show()

+----------+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+--------------------+-------------------+
|    LoanID|Age|Income|LoanAmount|CreditScore|MonthsEmployed|NumCreditLines|InterestRate|LoanTerm|DTIRatio|  Education|EmploymentType|MaritalStatus|HasMortgage|HasDependents|LoanPurpose|HasCoSigner|Default|             user_id|       txn_datetime|
+----------+---+------+----------+-----------+--------------+--------------+------------+--------+--------+-----------+--------------+-------------+-----------+-------------+-----------+-----------+-------+--------------------+-------------------+
|SC7OVY0X3Y| 62| 52384|     53174|        456|            87|             2|         6.6|      48|    0.58|High School|     Part-time|      Married|        Yes|           No|       Home|        Yes|      0|HlWpZydxscnmjGgeIPLi|2023-11-01 07:25:25|
|S172HKA

In [72]:
df.select('txn_datetime')

DataFrame[txn_datetime: string]

In [6]:
f_store.get_historical_features(
    entity_df=entity_df,
    features=['user_info:Age','user_info:Education']
).to_spark_df().show()

/opt/code/contrib/feast_epam/env/lib/python3.9/site-packages/feast/infra/offline_stores/contrib/spark_offline_store/spark_source.py:67: RuntimeWarning: The spark data source API is an experimental feature in alpha development. This API is unstable and it could and most probably will be changed in the future.
  warnings.warn(
/opt/code/contrib/feast_epam/env/lib/python3.9/site-packages/feast/infra/offline_stores/contrib/spark_offline_store/spark.py:137: RuntimeWarning: The spark offline store is an experimental feature in alpha development. Some functionality may still be unstable so functionality can change in the future.
  warnings.warn(


+--------------------+-------------------+---+-----------+
|             user_id|    event_timestamp|Age|  Education|
+--------------------+-------------------+---+-----------+
|HlWpZydxscnmjGgeIPLi|2023-01-02 07:00:00| 27|        PhD|
|RckfsuJrhXFYAToUVlmy|2024-01-09 07:00:00| 49|High School|
+--------------------+-------------------+---+-----------+

